# About this Notebook:

This notebook detailed the exploratory data analysis on the Open Images Classes. So we can prepared the ourself to how to use the data.

Notebook Enviroment

- Kernel: Python 3
- 

In [1]:
!python --version

Python 3.6.3 :: Anaconda custom (64-bit)


## Question to be answered:

1. How many classes can we use?
2. How many images perclass?

## TODO

# Load Modules

In [46]:
from __future__ import print_function
import pandas as pd
import numpy as np
import json
import os
from os.path import dirname as up
import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
%matplotlib inline

import seaborn as sns



# Commonly Shared Statics

In [47]:
data_labels_description_dir_path = '../../data/doc/labels/'

In [48]:
data_dir_path = '../../data/'

In [49]:
projectPath = up(up(os.getcwd()))

s3_bucket_name = "s3://insight-data-images/"

database_ini_file_path = "/utilities/database/database.ini"



In [50]:
print(projectPath)

/Users/zhonghengli/Documents/Insight/Project


# EDA on the class-descriptions.csv

In [51]:
class_descriptions_file_path = data_labels_description_dir_path + 'class-descriptions.csv'


In [52]:
df_class_descriptions = pd.read_csv(class_descriptions_file_path,names=['LabelName', 'Description'])

In [53]:
df_class_descriptions.head(10)

,LabelName,Description
0,/m/0100nhbf,Sprenger's tulip
1,/m/0104x9kv,Vinegret
2,/m/0105jzwx,Dabu-dabu
3,/m/0105ld7g,Pistachio ice cream
4,/m/0105lxy5,Woku
5,/m/0105n86x,Pastila
6,/m/0105ts35,Burasa
7,/m/0108_09c,Summer snowflake
8,/m/01_097,Airmail
9,/m/010dmf,Isle of man tt


In [54]:
df_class_descriptions.tail(10)

,LabelName,Description
19985,/m/0zdntfj,Candy cane sorrel
19986,/m/0zg7256,Gingerbread house
19987,/m/0zghq,Moonshine
19988,/m/0zkm,Arthropod
19989,/m/0zlm,Conservation-restoration
19990,/m/0zrpfhj,Throwing
19991,/m/0zrrls2,Red dahlia
19992,/m/0zrthkd,Brine
19993,/m/0zrv5th,Lamborghini huracán
19994,/m/0zvk5,Helmet


In [55]:
df_class_descriptions.describe()

,LabelName,Description
count,19995,19995
unique,19995,19712
top,/m/03h3knk,Trunk
freq,1,4


In [56]:
df_class_descriptions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19995 entries, 0 to 19994
Data columns (total 2 columns):
LabelName      19995 non-null object
Description    19995 non-null object
dtypes: object(2)
memory usage: 312.5+ KB


Replace space with underscore for S3 bucket prefix 

In [57]:
df_class_descriptions.Description = df_class_descriptions.Description.str.replace(' ', '_')

In [58]:
df_class_descriptions.tail(10)

,LabelName,Description
19985,/m/0zdntfj,Candy_cane_sorrel
19986,/m/0zg7256,Gingerbread_house
19987,/m/0zghq,Moonshine
19988,/m/0zkm,Arthropod
19989,/m/0zlm,Conservation-restoration
19990,/m/0zrpfhj,Throwing
19991,/m/0zrrls2,Red_dahlia
19992,/m/0zrthkd,Brine
19993,/m/0zrv5th,Lamborghini_huracán
19994,/m/0zvk5,Helmet


## Compose the list of 600 original labels 

In [59]:
! ls ../../data/doc/labels/



bbox_labels_600_hierarchy.json classes-trainable.txt
class-descriptions.csv         classes.txt


In [60]:
bbox_labels_600_hierarchy_json_file_path = data_labels_description_dir_path + 'bbox_labels_600_hierarchy.json'

In [61]:
bbox_labels_600_json_data=open(bbox_labels_600_hierarchy_json_file_path).read()

In [62]:
bbox_labels_600_json = json.loads(bbox_labels_600_json_data)

In [63]:
bbox_labels_600_json

{'LabelName': '/m/0bl9f',
 'Subcategory': [{'LabelName': '/m/0242l'},
  {'LabelName': '/m/03120'},
  {'LabelName': '/m/0h8l4fh'},
  {'LabelName': '/m/0138tl',
   'Subcategory': [{'LabelName': '/m/0167gd'},
    {'LabelName': '/m/01j51'},
    {'LabelName': '/m/029b3'},
    {'LabelName': '/m/02wmf'},
    {'LabelName': '/m/02zt3'},
    {'LabelName': '/m/0kmg4'}]},
  {'LabelName': '/m/019dx1',
   'Subcategory': [{'LabelName': '/m/0174k2'},
    {'LabelName': '/m/01k6s3'},
    {'LabelName': '/m/029bxz'},
    {'LabelName': '/m/02pjr4'},
    {'LabelName': '/m/02wv84t'},
    {'LabelName': '/m/02x984l'},
    {'LabelName': '/m/03qhv5'},
    {'LabelName': '/m/03s_tn'},
    {'LabelName': '/m/03wvsk'},
    {'LabelName': '/m/040b_t'},
    {'LabelName': '/m/04169hn'},
    {'LabelName': '/m/04lvq_'},
    {'LabelName': '/m/063rgb'},
    {'LabelName': '/m/07xyvk'},
    {'LabelName': '/m/0fx9l'},
    {'LabelName': '/m/0ky7b'},
    {'LabelName': '/m/0llzx'},
    {'LabelName': '/m/0c3m8g'},
    {'LabelName':

In [64]:
def item_generator(json_input, lookup_key):
    if isinstance(json_input, dict):
        for k, v in json_input.items():
            if k == lookup_key:
                yield v
            else:
                for child_val in item_generator(v, lookup_key):
                    yield child_val
    elif isinstance(json_input, list):
        for item in json_input:
            for item_val in item_generator(item, lookup_key):
                yield item_val

In [65]:
subcategory_set = set()
for label in item_generator(bbox_labels_600_json, "LabelName"):  
    subcategory_set.add(label)

In [66]:
subcategory_list =list(subcategory_set)

In [67]:
df_bbox_600_labels = pd.DataFrame(subcategory_list,columns=['LabelName'])

In [68]:
df_bbox_600_labels.head()

,LabelName
0,/m/02pdsw
1,/m/0zvk5
2,/m/02wv6h6
3,/m/0_cp5
4,/m/03rszm


In [69]:
df_bbox_600_labels.describe()

,LabelName
count,602
unique,602
top,/m/044r5d
freq,1


## Filter out 600 classes from class descriptions

In [70]:
df_trainables = pd.merge(df_bbox_600_labels,
                 df_class_descriptions[['LabelName', 'Description']],
                 on='LabelName', 
                 how='inner')

In [71]:
df_trainables.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 601 entries, 0 to 600
Data columns (total 2 columns):
LabelName      601 non-null object
Description    601 non-null object
dtypes: object(2)
memory usage: 14.1+ KB


In [72]:
df_trainables.head(10)

,LabelName,Description
0,/m/02pdsw,Cutting_board
1,/m/0zvk5,Helmet
2,/m/02wv6h6,Skirt
3,/m/0_cp5,Oyster
4,/m/03rszm,Curtain
5,/m/0152hh,Snowman
6,/m/0k65p,Human_hand
7,/m/0dtln,Lamp
8,/m/0hdln,Ruler
9,/m/0h99cwc,Kitchen_appliance


In [73]:
df_trainables.to_csv( data_dir_path + '600_trainable_LabelNames.csv', sep=',')

Listing all the labels 

In [74]:
for i in list(df_trainables['Description']):
    print (i)

Cutting_board
Helmet
Skirt
Oyster
Curtain
Snowman
Human_hand
Lamp
Ruler
Kitchen_appliance
Punching_bag
Harbor_seal
Bathtub
Butterfly
High_heels
Turtle
Headphones
Picnic_basket
Pineapple
Crown
Kangaroo
Pen
Spoon
Chisel
Apple
Camera
Bust
Watch
Eraser
Stretcher
Infant_bed
Magpie
Chair
Scissors
Spice_rack
Dishwasher
Cattle
Indoor_rower
Sculpture
Knife
Lynx
Dress
Accordion
Dolphin
Human_nose
Cupboard
Door
Filing_cabinet
Handgun
Sports_equipment
Candle
Rhinoceros
Aircraft
Ostrich
Stationary_bicycle
Sparrow
Tire
Lion
Tower
Fashion_accessory
Shirt
Hair_dryer
Cricket_ball
Porch
Flowerpot
Human_mouth
Office_building
Cabbage
Artichoke
Honeycomb
Watermelon
Radish
Land_vehicle
Drink
Salt_and_pepper_shakers
Wardrobe
Salad
Desk
Hiking_equipment
Skyscraper
Cream
Hedgehog
Raven
Tortoise
Coffee
Juice
Furniture
Beer
Mushroom
Alarm_clock
Window
Moths_and_butterflies
Giraffe
Dog
Kitchenware
Scoreboard
Palm_tree
Door_handle
Bathroom_cabinet
Cart
Trumpet
Cat_furniture
Insect
Wine
Kitchen_&_dining_room_table


### Create a dictionary from from the df_trainables

In [75]:
trainables_dict = df_trainables.set_index("LabelName").to_dict(orient="index")

In [76]:
trainables_dict

{'/m/011k07': {'Description': 'Tortoise'},
 '/m/011q46kg': {'Description': 'Container'},
 '/m/012074': {'Description': 'Magpie'},
 '/m/0120dh': {'Description': 'Sea_turtle'},
 '/m/01226z': {'Description': 'Football'},
 '/m/012n7d': {'Description': 'Ambulance'},
 '/m/012w5l': {'Description': 'Ladder'},
 '/m/012xff': {'Description': 'Toothbrush'},
 '/m/012ysf': {'Description': 'Syringe'},
 '/m/0130jx': {'Description': 'Sink'},
 '/m/0138tl': {'Description': 'Toy'},
 '/m/013y1f': {'Description': 'Organ'},
 '/m/01432t': {'Description': 'Cassette_deck'},
 '/m/014j1m': {'Description': 'Apple'},
 '/m/014sv8': {'Description': 'Human_eye'},
 '/m/014trl': {'Description': 'Cosmetics'},
 '/m/014y4n': {'Description': 'Paddle'},
 '/m/0152hh': {'Description': 'Snowman'},
 '/m/01599': {'Description': 'Beer'},
 '/m/015h_t': {'Description': 'Human_beard'},
 '/m/015p6': {'Description': 'Bird'},
 '/m/015qbp': {'Description': 'Parking_meter'},
 '/m/015qff': {'Description': 'Traffic_light'},
 '/m/015wgc': {'

In [77]:
for key, value in trainables_dict.items():
    print(value['Description'])

Cutting_board
Helmet
Skirt
Oyster
Curtain
Snowman
Human_hand
Lamp
Ruler
Kitchen_appliance
Punching_bag
Harbor_seal
Bathtub
Butterfly
High_heels
Turtle
Headphones
Picnic_basket
Pineapple
Crown
Kangaroo
Pen
Spoon
Chisel
Apple
Camera
Bust
Watch
Eraser
Stretcher
Infant_bed
Magpie
Chair
Scissors
Spice_rack
Dishwasher
Cattle
Indoor_rower
Sculpture
Knife
Lynx
Dress
Accordion
Dolphin
Human_nose
Cupboard
Door
Filing_cabinet
Handgun
Sports_equipment
Candle
Rhinoceros
Aircraft
Ostrich
Stationary_bicycle
Sparrow
Tire
Lion
Tower
Fashion_accessory
Shirt
Hair_dryer
Cricket_ball
Porch
Flowerpot
Human_mouth
Office_building
Cabbage
Artichoke
Honeycomb
Watermelon
Radish
Land_vehicle
Drink
Salt_and_pepper_shakers
Wardrobe
Salad
Desk
Hiking_equipment
Skyscraper
Cream
Hedgehog
Raven
Tortoise
Coffee
Juice
Furniture
Beer
Mushroom
Alarm_clock
Window
Moths_and_butterflies
Giraffe
Dog
Kitchenware
Scoreboard
Palm_tree
Door_handle
Bathroom_cabinet
Cart
Trumpet
Cat_furniture
Insect
Wine
Kitchen_&_dining_room_table


Adding label '/m/0bl9f' with description "Entity"  

In [78]:
trainables_dict['/m/0bl9f'] = {'Description': 'Entity'}

In [79]:
trainables_dict['/m/0bl9f']

{'Description': 'Entity'}

In [80]:
trainables_dict['/m/0152hh']

{'Description': 'Snowman'}

## Convert labels to human readable description

In [83]:
bbox_labels_600_json = json.loads(bbox_labels_600_json_data)

In [84]:
def label_to_description(json_input, lookup_key):
    if isinstance(json_input, dict):
        for k, v in json_input.items():
            if k == lookup_key:
                json_input[k] = trainables_dict[v]['Description']
 
                yield v
            else:
                for child_val in label_to_description(v, lookup_key):
                      
                    yield child_val
    elif isinstance(json_input, list):
        for item in json_input:
            for item_val in label_to_description(item, lookup_key):
                yield item_val

In [85]:
for label in label_to_description(bbox_labels_600_json, "LabelName"):  
    print("")

# label_to_description(bbox_labels_600_json, "LabelName")

In [86]:
bbox_labels_600_json

{'LabelName': 'Entity',
 'Subcategory': [{'LabelName': 'Coin'},
  {'LabelName': 'Flag'},
  {'LabelName': 'Light_bulb'},
  {'LabelName': 'Toy',
   'Subcategory': [{'LabelName': 'Doll'},
    {'LabelName': 'Balloon'},
    {'LabelName': 'Dice'},
    {'LabelName': 'Flying_disc'},
    {'LabelName': 'Kite'},
    {'LabelName': 'Teddy_bear'}]},
  {'LabelName': 'Home_appliance',
   'Subcategory': [{'LabelName': 'Washing_machine'},
    {'LabelName': 'Toaster'},
    {'LabelName': 'Oven'},
    {'LabelName': 'Blender'},
    {'LabelName': 'Gas_stove'},
    {'LabelName': 'Mechanical_fan'},
    {'LabelName': 'Heater'},
    {'LabelName': 'Kettle'},
    {'LabelName': 'Hair_dryer'},
    {'LabelName': 'Refrigerator'},
    {'LabelName': 'Wood-burning_stove'},
    {'LabelName': 'Humidifier'},
    {'LabelName': 'Mixer'},
    {'LabelName': 'Coffeemaker'},
    {'LabelName': 'Microwave_oven'},
    {'LabelName': 'Dishwasher'},
    {'LabelName': 'Sewing_machine'},
    {'LabelName': 'Hand_dryer'},
    {'LabelName':

## EDA on the hierchy of 600 original labels by classes¶

In [87]:
bbox_labels_600_json

{'LabelName': 'Entity',
 'Subcategory': [{'LabelName': 'Coin'},
  {'LabelName': 'Flag'},
  {'LabelName': 'Light_bulb'},
  {'LabelName': 'Toy',
   'Subcategory': [{'LabelName': 'Doll'},
    {'LabelName': 'Balloon'},
    {'LabelName': 'Dice'},
    {'LabelName': 'Flying_disc'},
    {'LabelName': 'Kite'},
    {'LabelName': 'Teddy_bear'}]},
  {'LabelName': 'Home_appliance',
   'Subcategory': [{'LabelName': 'Washing_machine'},
    {'LabelName': 'Toaster'},
    {'LabelName': 'Oven'},
    {'LabelName': 'Blender'},
    {'LabelName': 'Gas_stove'},
    {'LabelName': 'Mechanical_fan'},
    {'LabelName': 'Heater'},
    {'LabelName': 'Kettle'},
    {'LabelName': 'Hair_dryer'},
    {'LabelName': 'Refrigerator'},
    {'LabelName': 'Wood-burning_stove'},
    {'LabelName': 'Humidifier'},
    {'LabelName': 'Mixer'},
    {'LabelName': 'Coffeemaker'},
    {'LabelName': 'Microwave_oven'},
    {'LabelName': 'Dishwasher'},
    {'LabelName': 'Sewing_machine'},
    {'LabelName': 'Hand_dryer'},
    {'LabelName':

In [103]:
for i in bbox_labels_600_json.items():
    print(i)

('LabelName', 'Entity')
('Subcategory', [{'LabelName': 'Coin'}, {'LabelName': 'Flag'}, {'LabelName': 'Light_bulb'}, {'LabelName': 'Toy', 'Subcategory': [{'LabelName': 'Doll'}, {'LabelName': 'Balloon'}, {'LabelName': 'Dice'}, {'LabelName': 'Flying_disc'}, {'LabelName': 'Kite'}, {'LabelName': 'Teddy_bear'}]}, {'LabelName': 'Home_appliance', 'Subcategory': [{'LabelName': 'Washing_machine'}, {'LabelName': 'Toaster'}, {'LabelName': 'Oven'}, {'LabelName': 'Blender'}, {'LabelName': 'Gas_stove'}, {'LabelName': 'Mechanical_fan'}, {'LabelName': 'Heater'}, {'LabelName': 'Kettle'}, {'LabelName': 'Hair_dryer'}, {'LabelName': 'Refrigerator'}, {'LabelName': 'Wood-burning_stove'}, {'LabelName': 'Humidifier'}, {'LabelName': 'Mixer'}, {'LabelName': 'Coffeemaker'}, {'LabelName': 'Microwave_oven'}, {'LabelName': 'Dishwasher'}, {'LabelName': 'Sewing_machine'}, {'LabelName': 'Hand_dryer'}, {'LabelName': 'Ceiling_fan'}]}, {'LabelName': 'Plumbing_fixture', 'Subcategory': [{'LabelName': 'Sink'}, {'LabelName': 

In [98]:
def hierchy_generator(json_input, lookup_key, parent_key):
    if isinstance(json_input, dict):
        
        new_parent_key = json_input[lookup_key]
        
        print("new_parent_key: ", new_parent_key)

        for k, v in json_input.items():
 
            if k == lookup_key:

                yield v
                
            else:
                for child_val in hierchy_generator(v, lookup_key, new_parent_key):

                    yield child_val
    elif isinstance(json_input, list):
        for item in json_input:
            for item_val in hierchy_generator(item, lookup_key, parent_key):
                yield item_val

In [99]:
print(type(bbox_labels_600_json))

<class 'dict'>


In [100]:
classes_hierchy_dict = {}

for label in hierchy_generator(bbox_labels_600_json, "LabelName", None):  
    
    print("Do nothing")

new_parent_key:  Entity
Do nothing
new_parent_key:  Coin
Do nothing
new_parent_key:  Flag
Do nothing
new_parent_key:  Light_bulb
Do nothing
new_parent_key:  Toy
Do nothing
new_parent_key:  Doll
Do nothing
new_parent_key:  Balloon
Do nothing
new_parent_key:  Dice
Do nothing
new_parent_key:  Flying_disc
Do nothing
new_parent_key:  Kite
Do nothing
new_parent_key:  Teddy_bear
Do nothing
new_parent_key:  Home_appliance
Do nothing
new_parent_key:  Washing_machine
Do nothing
new_parent_key:  Toaster
Do nothing
new_parent_key:  Oven
Do nothing
new_parent_key:  Blender
Do nothing
new_parent_key:  Gas_stove
Do nothing
new_parent_key:  Mechanical_fan
Do nothing
new_parent_key:  Heater
Do nothing
new_parent_key:  Kettle
Do nothing
new_parent_key:  Hair_dryer
Do nothing
new_parent_key:  Refrigerator
Do nothing
new_parent_key:  Wood-burning_stove
Do nothing
new_parent_key:  Humidifier
Do nothing
new_parent_key:  Mixer
Do nothing
new_parent_key:  Coffeemaker
Do nothing
new_parent_key:  Microwave_oven

new_parent_key:  Snowboard
Do nothing
new_parent_key:  Skateboard
Do nothing
new_parent_key:  Ski
Do nothing
new_parent_key:  Bowling_equipment
Do nothing
new_parent_key:  Dumbbell
Do nothing
new_parent_key:  Stationary_bicycle
Do nothing
new_parent_key:  Treadmill
Do nothing
new_parent_key:  Training_bench
Do nothing
new_parent_key:  Indoor_rower
Do nothing
new_parent_key:  Horizontal_bar
Do nothing
new_parent_key:  Parachute
Do nothing
new_parent_key:  Racket
Do nothing
new_parent_key:  Tennis_racket
Do nothing
new_parent_key:  Table_tennis_racket
Do nothing
new_parent_key:  Balance_beam
Do nothing
new_parent_key:  Helmet
Do nothing
new_parent_key:  Bicycle_helmet
Do nothing
new_parent_key:  Football_helmet
Do nothing
new_parent_key:  Billiard_table
Do nothing
new_parent_key:  Tool
Do nothing
new_parent_key:  Container
Do nothing
new_parent_key:  Tin_can
Do nothing
new_parent_key:  Barrel
Do nothing
new_parent_key:  Bottle
Do nothing
new_parent_key:  Picnic_basket
Do nothing
new_pare

Do nothing
new_parent_key:  Turtle
Do nothing
new_parent_key:  Tortoise
Do nothing
new_parent_key:  Sea_turtle
Do nothing
new_parent_key:  Crocodile
Do nothing
new_parent_key:  Frog
Do nothing
new_parent_key:  Fish
Do nothing
new_parent_key:  Goldfish
Do nothing
new_parent_key:  Shark
Do nothing
new_parent_key:  Rays_and_skates
Do nothing
new_parent_key:  Seahorse
Do nothing
new_parent_key:  Shellfish
Do nothing
new_parent_key:  Oyster
Do nothing
new_parent_key:  Lobster
Do nothing
new_parent_key:  Shrimp
Do nothing
new_parent_key:  Crab
Do nothing
new_parent_key:  Cosmetics
Do nothing
new_parent_key:  Face_powder
Do nothing
new_parent_key:  Hair_spray
Do nothing
new_parent_key:  Lipstick
Do nothing
new_parent_key:  Perfume
Do nothing
new_parent_key:  Personal_care
Do nothing
new_parent_key:  Toothbrush
Do nothing
new_parent_key:  Sunglasses
Do nothing
new_parent_key:  Goggles
Do nothing
new_parent_key:  Crutch
Do nothing
new_parent_key:  Cream
Do nothing
new_parent_key:  Diaper
Do not

In [101]:
classes_hierchy_dict

{}

In [60]:
classes_hierchy_dict['Accordion']['parent']

KeyError: 'Accordion'

## EDA on the classes-trainable.txt

In [ ]:
classes_trainable_file_path = data_labels_description_dir_path + 'classes-trainable.txt'



In [ ]:
df_classes_trainable = pd.read_csv(classes_trainable_file_path,names=['LabelName'])

In [ ]:
df_classes_trainable.head(10)

In [ ]:
df_classes_trainable.describe()

In [ ]:
df_classes_trainable.info()

## Filter out trainable classes from class descriptions

In [ ]:
df_trainables = pd.merge(df_classes_trainable,
                 df_class_descriptions[['LabelName', 'Description']],
                 on='LabelName', 
                 how='inner')

In [ ]:
df_trainables.info()

In [ ]:
df_trainables.head()

## EDA on train-annotations-human-imagelabels.csv

In [ ]:
train_annotations_human_imagelabels_file_path = data_labels_description_dir_path + 'train-annotations-human-imagelabels.csv'

In [ ]:
df_train_annotations_human_imagelabels = pd.read_csv(train_annotations_human_imagelabels_file_path,header=0)

In [ ]:
df_train_annotations_human_imagelabels.info()

In [ ]:
df_train_annotations_human_imagelabels.head()

In [ ]:
df_train_annotations_human_imagelabels_confident = df_train_annotations_human_imagelabels[df_train_annotations_human_imagelabels['Confidence']== 1
                                                                                         ]

In [ ]:
df_train_annotations_human_imagelabels_confident.info()

In [ ]:
df_train_annotations_human_imagelabels_confident.head(20)

In [ ]:
df_train_annotations_human_imagelabels_imgIDs_Labels = df_train_annotations_human_imagelabels[['ImageID','LabelName']]

In [ ]:
df_train_annotations_human_imagelabels_imgIDs_Labels.head()

## Mergin with ImageIDs

In [ ]:
df_trainables_imageIds_label_descriptions = pd.merge(df_train_annotations_human_imagelabels_imgIDs_Labels,
                 df_trainables[['LabelName', 'Description']],
                 on='LabelName', 
                 how='inner')

In [ ]:
df_trainables_imageIds_label_descriptions.info()

In [ ]:
df_trainables_imageIds_label_descriptions.head(100)

In [ ]:
df_trainables_imageIds_label_descriptions.to_csv( data_dir_path + 'imageIDs_LabelNames.csv', sep=',')

## EDA on train-annotations-machine-imagelabels.csv

In [ ]:
train_annotations_machine_imagelabels_file_path = data_labels_description_dir_path + 'train-annotations-machine-imagelabels.csv'

In [ ]:
df_train_annotations_machine_imagelabels = pd.read_csv(train_annotations_machine_imagelabels_file_path,header=0)

In [ ]:
df_train_annotations_machine_imagelabels.head()

In [ ]:
## EDA on train-annotations-human-imagelabels.csv

In [88]:
bbox_labels_600_json


{'LabelName': 'Entity',
 'Subcategory': [{'LabelName': 'Coin'},
  {'LabelName': 'Flag'},
  {'LabelName': 'Light_bulb'},
  {'LabelName': 'Toy',
   'Subcategory': [{'LabelName': 'Doll'},
    {'LabelName': 'Balloon'},
    {'LabelName': 'Dice'},
    {'LabelName': 'Flying_disc'},
    {'LabelName': 'Kite'},
    {'LabelName': 'Teddy_bear'}]},
  {'LabelName': 'Home_appliance',
   'Subcategory': [{'LabelName': 'Washing_machine'},
    {'LabelName': 'Toaster'},
    {'LabelName': 'Oven'},
    {'LabelName': 'Blender'},
    {'LabelName': 'Gas_stove'},
    {'LabelName': 'Mechanical_fan'},
    {'LabelName': 'Heater'},
    {'LabelName': 'Kettle'},
    {'LabelName': 'Hair_dryer'},
    {'LabelName': 'Refrigerator'},
    {'LabelName': 'Wood-burning_stove'},
    {'LabelName': 'Humidifier'},
    {'LabelName': 'Mixer'},
    {'LabelName': 'Coffeemaker'},
    {'LabelName': 'Microwave_oven'},
    {'LabelName': 'Dishwasher'},
    {'LabelName': 'Sewing_machine'},
    {'LabelName': 'Hand_dryer'},
    {'LabelName':

In [89]:
def hierchy_generator(json_input, lookup_key, parent_key):
    if isinstance(json_input, dict):
        
        new_parent_key = json_input[lookup_key]
        
        # ini new key
        print("new_parent_key: ", new_parent_key)
        classes_hierchy_dict[new_parent_key] = {}
        classes_hierchy_dict[new_parent_key]['children'] = []
        classes_hierchy_dict[new_parent_key]['parent'] = ""

        for k, v in json_input.items():
 
            if k == lookup_key:
                classes_hierchy_dict[v]['parent'] = parent_key
                yield v
                
            else:
                for child_val in hierchy_generator(v, lookup_key, new_parent_key):

                    yield child_val
    elif isinstance(json_input, list):

        for item in json_input:
            for item_val in hierchy_generator(item, lookup_key, parent_key):
                
                classes_hierchy_dict[parent_key]['children'].append(item_val)

                yield item_val

In [90]:
classes_hierchy_dict = {}
for label in hierchy_generator(bbox_labels_600_json, "LabelName", None):  
    pass

new_parent_key:  Entity
new_parent_key:  Coin
new_parent_key:  Flag
new_parent_key:  Light_bulb
new_parent_key:  Toy
new_parent_key:  Doll
new_parent_key:  Balloon
new_parent_key:  Dice
new_parent_key:  Flying_disc
new_parent_key:  Kite
new_parent_key:  Teddy_bear
new_parent_key:  Home_appliance
new_parent_key:  Washing_machine
new_parent_key:  Toaster
new_parent_key:  Oven
new_parent_key:  Blender
new_parent_key:  Gas_stove
new_parent_key:  Mechanical_fan
new_parent_key:  Heater
new_parent_key:  Kettle
new_parent_key:  Hair_dryer
new_parent_key:  Refrigerator
new_parent_key:  Wood-burning_stove
new_parent_key:  Humidifier
new_parent_key:  Mixer
new_parent_key:  Coffeemaker
new_parent_key:  Microwave_oven
new_parent_key:  Dishwasher
new_parent_key:  Sewing_machine
new_parent_key:  Hand_dryer
new_parent_key:  Ceiling_fan
new_parent_key:  Plumbing_fixture
new_parent_key:  Sink
new_parent_key:  Bidet
new_parent_key:  Shower
new_parent_key:  Tap
new_parent_key:  Bathtub
new_parent_key:  To

new_parent_key:  Human_body
new_parent_key:  Human_eye
new_parent_key:  Skull
new_parent_key:  Human_head
new_parent_key:  Human_face
new_parent_key:  Human_mouth
new_parent_key:  Human_ear
new_parent_key:  Human_nose
new_parent_key:  Human_hair
new_parent_key:  Human_hand
new_parent_key:  Human_foot
new_parent_key:  Human_arm
new_parent_key:  Human_leg
new_parent_key:  Human_beard
new_parent_key:  Man
new_parent_key:  Woman
new_parent_key:  Boy
new_parent_key:  Girl
new_parent_key:  Food
new_parent_key:  Fast_food
new_parent_key:  Hot_dog
new_parent_key:  French_fries
new_parent_key:  Waffle
new_parent_key:  Pancake
new_parent_key:  Burrito
new_parent_key:  Snack
new_parent_key:  Pretzel
new_parent_key:  Popcorn
new_parent_key:  Cookie
new_parent_key:  Dessert
new_parent_key:  Muffin
new_parent_key:  Cookie
new_parent_key:  Ice_cream
new_parent_key:  Cake
new_parent_key:  Candy
new_parent_key:  Guacamole
new_parent_key:  Fruit
new_parent_key:  Apple
new_parent_key:  Grape
new_parent_k

In [91]:
classes_hierchy_dict

{'Accordion': {'children': [], 'parent': 'Musical_instrument'},
 'Adhesive_tape': {'children': [], 'parent': 'Office_supplies'},
 'Aircraft': {'children': ['Helicopter', 'Airplane', 'Rocket'],
  'parent': 'Vehicle'},
 'Airplane': {'children': [], 'parent': 'Aircraft'},
 'Alarm_clock': {'children': [], 'parent': 'Clock'},
 'Alpaca': {'children': [], 'parent': 'Mammal'},
 'Ambulance': {'children': [], 'parent': 'Land_vehicle'},
 'Animal': {'children': ['Bird',
   'Magpie',
   'Woodpecker',
   'Blue_jay',
   'Ostrich',
   'Penguin',
   'Raven',
   'Chicken',
   'Eagle',
   'Owl',
   'Duck',
   'Canary',
   'Goose',
   'Swan',
   'Falcon',
   'Parrot',
   'Sparrow',
   'Turkey',
   'Invertebrate',
   'Tick',
   'Centipede',
   'Marine_invertebrates',
   'Starfish',
   'Isopod',
   'Squid',
   'Lobster',
   'Jellyfish',
   'Shrimp',
   'Crab',
   'Insect',
   'Bee',
   'Beetle',
   'Ladybug',
   'Ant',
   'Moths_and_butterflies',
   'Caterpillar',
   'Butterfly',
   'Dragonfly',
   'Scorpio

In [92]:
for k, c in classes_hierchy_dict.items():
    print("key: ", k)
    print("parent: ", c['parent'])
#     print(c['children'])

key:  Entity
parent:  None
key:  Coin
parent:  Entity
key:  Flag
parent:  Entity
key:  Light_bulb
parent:  Entity
key:  Toy
parent:  Entity
key:  Doll
parent:  Toy
key:  Balloon
parent:  Toy
key:  Dice
parent:  Toy
key:  Flying_disc
parent:  Sports_equipment
key:  Kite
parent:  Toy
key:  Teddy_bear
parent:  Bear
key:  Home_appliance
parent:  Entity
key:  Washing_machine
parent:  Home_appliance
key:  Toaster
parent:  Home_appliance
key:  Oven
parent:  Kitchen_appliance
key:  Blender
parent:  Kitchen_appliance
key:  Gas_stove
parent:  Home_appliance
key:  Mechanical_fan
parent:  Home_appliance
key:  Heater
parent:  Home_appliance
key:  Kettle
parent:  Home_appliance
key:  Hair_dryer
parent:  Home_appliance
key:  Refrigerator
parent:  Kitchen_appliance
key:  Wood-burning_stove
parent:  Home_appliance
key:  Humidifier
parent:  Home_appliance
key:  Mixer
parent:  Kitchen_appliance
key:  Coffeemaker
parent:  Kitchen_appliance
key:  Microwave_oven
parent:  Kitchen_appliance
key:  Dishwasher
p

parent:  Fish
key:  Seahorse
parent:  Fish
key:  Cosmetics
parent:  Entity
key:  Face_powder
parent:  Cosmetics
key:  Hair_spray
parent:  Cosmetics
key:  Lipstick
parent:  Cosmetics
key:  Perfume
parent:  Cosmetics
key:  Personal_care
parent:  Entity
key:  Crutch
parent:  Personal_care
key:  Cream
parent:  Personal_care
key:  Diaper
parent:  Personal_care
key:  Drink
parent:  Entity
key:  Beer
parent:  Drink
key:  Cocktail
parent:  Drink
key:  Coffee
parent:  Drink
key:  Juice
parent:  Drink
key:  Tea
parent:  Drink
key:  Wine
parent:  Drink
key:  Door_handle
parent:  Door
key:  Door
parent:  Entity
key:  Window
parent:  Entity
key:  Stairs
parent:  Entity
key:  Porch
parent:  Entity
key:  Coconut
parent:  Entity


## uploading the labels into DB

In [93]:
! pwd

/Users/zhonghengli/Documents/Insight/Project/doc/noteBooks


In [94]:
import psycopg2
from psycopg2 import extras
from configparser import ConfigParser
import datetime

In [97]:

"""

config Database

"""

def config(filename=projectPath+database_ini_file_path, section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db



In [98]:
sql = "INSERT INTO labels (label_name, parent_name, children, image_count,updated_date) VALUES %s ;"


""" Connect to the PostgreSQL database server """
conn = None
try:
    # read connection parameters
    params = config()

    # connect to the PostgreSQL server
    print('Connecting to the PostgreSQL database...')
    conn = psycopg2.connect(**params)

    # create a cursor
    cur = conn.cursor()

    # TODO -  augmented SQL statement
    values_list = []
    
    for key, value in classes_hierchy_dict.items():

        values = (
                    key,
                    value['parent'],
                    value['children'],
                    0,
                    datetime.datetime.now(),
                      )

        values_list.append(values)

    # writing image info into the database
    # execute a statement
    print('writing image batch info into the database...')
    psycopg2.extras.execute_values(cur, sql, values_list)
    
    # alter table labels to add self-reference FOREIGN KEY
    alter_sql = " ALTER TABLE labels ADD FOREIGN KEY (parent_name) REFERENCES labels(label_name);"
    
    
    # commit the changes to the database
    conn.commit()

    batch_id = cur.fetchone()[0]
    # close the communication with the PostgreSQL
    cur.close()


except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
        print('Database connection closed.')



Connecting to the PostgreSQL database...
writing image batch info into the database...
no results to fetch
Database connection closed.
